In [ ]:
from forrestprf import pupil, data
import seaborn as sns
import numpy as np
import nibabel as nib
from matplotlib import pyplot as plt
from datamatrix import DataMatrix, io, functional as fnc, NiftiColumn, FloatColumn
from scipy import stats
import multiprocessing

DT = 2
RUNS = 1, 2, 3, 4, 5, 6, 7, 8

In [ ]:
def trim(a, minval=-np.inf, maxval=np.inf):
    
    a[(a < minval) | (a > maxval)] = np.nan
    
    
dm = io.readpickle('outputs/prf-dm.pkl')
for row in dm:    
    trim(row.prf_x.get_data(), minval=4, maxval=156)
    trim(row.prf_y.get_data(), minval=4, maxval=124)
    trim(row.prf_sd.get_data(), minval=1, maxval=60)

In [ ]:
def do_subject(args):
    
    sub, roi, xyz = args
    rdm = DataMatrix(length=len(RUNS))
    rdm.r_vc_pupil = NiftiColumn
    rdm.r_vc_lc = NiftiColumn
    rdm.r_lc_pupil = FloatColumn
    rdm.sub = sub
    rdm.roi = roi
    for row, run in zip(rdm, RUNS):
        print(sub, roi, run)
        sub_data = pupil.avmovie_data(sub, run)
        pupil_data = pupil.pupil_data(sub, run)
        lc_data = pupil.lc_data(sub, run)
        row.r_vc_pupil = pupil.pupil_cor_img(
            sub_data,
            pupil_data,
            xyz,
            dt=DT
        )
        row.r_vc_lc = pupil.pupil_cor_img(
            sub_data,
            lc_data,
            xyz,
            dt=0
        )
        s, i, r, p, se = stats.linregress(lc_data[DT:], pupil_data[:-DT])
        row.r_lc_pupil = r
    return rdm


args = [
    (
        row.sub,
        row.roi,
        np.where(~np.isnan(row.prf_x.get_data()))
    ) for row in dm
]
with multiprocessing.Pool(2) as pool:
    results = pool.map(do_subject, args)

In [ ]:
dm.r_vc_pupil = NiftiColumn
dm.r_vc_lc = NiftiColumn
dm.r_lc_pupil = FloatColumn
for rdm in results:
    i = (dm.roi == rdm.roi[0]) & (dm.sub == rdm.sub[0])
    dm.r_vc_pupil[i] = rdm.r_vc_pupil.mean
    dm.r_vc_lc[i] = rdm.r_vc_lc.mean
    dm.r_lc_pupil[i] = rdm.r_lc_pupil.mean
csvdm = dm[:]
del csvdm.prf_err
del csvdm.prf_sd
del csvdm.prf_x
del csvdm.prf_y
del csvdm.r_vc_pupil
del csvdm.r_vc_lc
io.writetxt(csvdm, 'outputs/r_pupil.csv')
io.writepickle(dm, 'outputs/r_pupil.pkl')

# Overall correlation per ROI

In [ ]:
import seaborn as sns

dm.r = [np.nanmean(row.r_pupil.get_data()) for row in dm]
sns.violinplot(x='roi', y='r', data=dm)
plt.axhline(0)

In [ ]:
from scipy import stats
from datamatrix import operations as ops
for roi, rdm in ops.split(dm.roi):
    t, p = stats.ttest_1samp(rdm.r, 0)
    print('{}: t({}) = {:.2f}, p = {:4f}'.format(roi, len(rdm) - 1, t, p)) 

# Convert to longish format

In [ ]:
def flatten(nft):
    
    a = nft.get_data().flatten()
    return a[~np.isnan(a)]

    
ldm = DataMatrix()
for row in dm:
    x = flatten(row.prf_x)
    y = flatten(row.prf_y)
    sd = flatten(row.prf_sd)
    err = flatten(row.prf_err)
    r = flatten(row.r_pupil)
    sdm = DataMatrix(len(x))
    sdm.sub = row.sub
    sdm.roi = row.roi
    sdm.r_pupil = FloatColumn
    sdm.prf_x = FloatColumn
    sdm.prf_y = FloatColumn
    sdm.prf_sd = FloatColumn
    sdm.r_pupil = r
    sdm.prf_x = x
    sdm.prf_y = y
    sdm.prf_sd = sd
    ldm <<= sdm
ldm.ecc = ((ldm.prf_x - 80) ** 2 + (ldm.prf_y - 64)) ** .5
ldm

# Correlations with RF properties

In [ ]:
def bin_plot(dm, x, y, roi, bins=3):

    lx = []
    ly = []
    lmin = []
    lmax = []
    for bdm in ops.bin_split(dm[x], bins=bins):
        lx.append(bdm[x].mean)
        ly.append(bdm[y].mean)
        lmin.append(bdm[y].mean - bdm[y].std / len(bdm[y]) ** .5)
        lmax.append(bdm[y].mean + bdm[y].std / len(bdm[y]) ** .5)
    plt.fill_between(lx, lmin, lmax, alpha=.25)
    plt.plot(lx, ly, 'o-', label=roi)
    

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
for i, (roi, rdm) in enumerate(ops.split(ldm.roi)):
    bin_plot(rdm, 'ecc', 'r_pupil', roi)
bin_plot(ldm, 'ecc', 'r_pupil', 'overall')
plt.xlabel('Eccentricity')
plt.ylim(-.04, -.01)
plt.legend()
plt.subplot(1, 2, 2)
for i, (roi, rdm) in enumerate(ops.split(ldm.roi)):
    bin_plot(rdm, 'prf_sd', 'r_pupil', roi)
bin_plot(ldm, 'prf_sd', 'r_pupil', 'overall')
plt.xlabel('RF Size')
plt.ylim(-.04, -.01)
plt.legend()
plt.show()

In [ ]:
from datamatrix.rbridge import lme4

lm = lme4.lmer(ldm, 'r_pupil ~ ecc * roi + (1 + ecc * roi|sub)')
print(lm)
lm = lme4.lmer(ldm, 'r_pupil ~ prf_sd * roi + (1 + prf_sd * roi|sub)')
print(lm)